In [20]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [21]:
#ref: Solving two-stage robust optimization problems using a column-and-constraint generation method

In [22]:
#define master problem

In [8]:

# Constant creation
f = [400, 414, 326]
a = [18, 25, 20]
C = [[22, 33, 24],
      [33, 23, 30],
      [20, 25, 27], ]
d = [206, 274, 220]
k = 0  # Iterative counting

# Create model
MP = gp.Model()  # Master-problem
SP = gp.Model()  # Sub-problem(KKT)
SPSD = gp.Model()  # Sub-problem (strong duality)
# Construction of Master-problem
# addVars
x = MP.addVars(3, 3, lb=0, obj=C, vtype=GRB.CONTINUOUS, name='x')
y = MP.addVars(len(f), lb=0, ub=1, obj=f, vtype=GRB.INTEGER, name='y')
z = MP.addVars(len(a), lb=0, obj=a, vtype=GRB.CONTINUOUS, name='z')

# addConstrs
Column1 = MP.addConstrs((z[i] <= 800 * y[i] for i in range(3)), name='fixCap')
Column4 = MP.addConstr(gp.quicksum(z[i] for i in range(3)) >= 700, name='zCap') #772=206+274+220+40*1.8
S1 = MP.addConstrs(((gp.quicksum(x[i, j] for j in range(3))) <= z[i] for i in range(3)), name='zCap')
S2 = MP.addConstrs(((gp.quicksum(x[i, j] for i in range(3))) >= d[j] for j in range(3)), name='demand')


#MP.Params.timeLimit = 100.0 # 等价于 MP.setParam('timeLimit', 100)
#MP.Params.InfUnbdInfo = 1  # Determines whether simplex (and crossover) will compute additional information when a model is determined to be infeasible or unbounded
#设置是否在控制台显示优化信息：
MP.Params.LogToConsole = 0
#设置线性规划的求解方法：
#MP.params.Method = 1  # 使用对偶单纯形法
MP.params.Method = 0  # 使用原始单纯形法(迭代慢，但占内存小）
#MP.params.Method = 2  # 使用内点法（gurobi称作barrier法）
#检查约束条件，模型得不到可行解时，才能用这个函数
#使用 MP.computeIIS() 检查不可行的约束条件
#MP.computeIIS()
#MP.write("MPilp.ilp")
#使用 m.feasRelax() 通过松弛最少的不可行约束条件，得到一个可行解（模型得不到可行解时，才能用这个函数）
if MP.Status == 3: # model is infeasible 
    print("Model is infeasible")
    MP.computeIIS()
    MP.write("model.ilp")
#print(MP.getConstrs())
MP.write("MPDeter.lp")  # model print and visual inspection model,can open it with Notepad++
MP.optimize()  # Solve Model
LB = MP.objval  # get optimum value of model
print('Optimal value '+str(MP.ObjVal))
print('Optimal solution', end = " ")
for i in MP.getVars():
    print('%s = %g' % (i.varName, i.x), end = " ")
print()
print('LB: '+str(LB))
        

Optimal value 30536.0
Optimal solution x[0,0] = 206 x[0,1] = 0 x[0,2] = 220 x[1,0] = 0 x[1,1] = 0 x[1,2] = 0 x[2,0] = 0 x[2,1] = 274 x[2,2] = 0 y[0] = 1 y[1] = 0 y[2] = 1 z[0] = 426 z[1] = 0 z[2] = 274 
LB: 30536.0
